### Mount gdrive

### This cell is for Google colaboratory users.

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%cd /gdrive/My\ Drive/DNNTopology

### Parameter setting

In [0]:
class_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
firstLayerSize = 300
secondLayerSize = 100
outputNeuron = 10

outputDir = "./models/"
epochs = 30
batch_size = 32
model_name = outputDir + "firstLayerSize"+str(firstLayerSize)+"secondLayerSize"+str(secondLayerSize)+"outputNeuron"+str(outputNeuron)+ "class" + str(class_list) + ".h5"

### Import libraries

In [0]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
%matplotlib inline

### Load CIFAR10 data set

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Extract data in class_list
train_filter = np.where( [ (element in class_list) for element in y_train] )
test_filter = np.where( [ (element in class_list) for element in y_test] )
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

In [0]:
# Normalize image data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# Convert vectors to binary class
y_train = keras.utils.to_categorical(y_train, outputNeuron)
y_test = keras.utils.to_categorical(y_test, outputNeuron)

### Make model

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:], activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(firstLayerSize, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(secondLayerSize, activation = 'relu'))
model.add(Dense(outputNeuron, activation = 'softmax'))

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 64)         

### Train model

In [0]:
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
fit = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 10s 206us/step - loss: 1.9000 - acc: 0.2947 - val_loss: 1.6759 - val_acc: 0.3872
Epoch 2/30
50000/50000 [==============================] - 10s 199us/step - loss: 1.5598 - acc: 0.4301 - val_loss: 1.4087 - val_acc: 0.4880
Epoch 3/30
50000/50000 [==============================] - 10s 194us/step - loss: 1.4187 - acc: 0.4889 - val_loss: 1.2897 - val_acc: 0.5373
Epoch 4/30
50000/50000 [==============================] - 10s 198us/step - loss: 1.3150 - acc: 0.5296 - val_loss: 1.2647 - val_acc: 0.5470
Epoch 5/30
50000/50000 [==============================] - 10s 191us/step - loss: 1.2296 - acc: 0.5636 - val_loss: 1.1064 - val_acc: 0.6058
Epoch 6/30
50000/50000 [==============================] - 10s 200us/step - loss: 1.1574 - acc: 0.5917 - val_loss: 1.0430 - val_acc: 0.6241
Epoch 7/30
50000/50000 [==============================] - 10s 192us/step - loss: 1.1015 - acc: 0.6113 - val_loss: 1.

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 1s 68us/step
Test loss: 0.6814553399085999
Test accuracy: 0.7725


### Save model

In [0]:
model.save(model_name, include_optimizer = False)